In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X = np.genfromtxt('data/X.csv', delimiter=',')
Y = np.genfromtxt('data/Y.csv', delimiter=',')

In [3]:
X.shape

(99990, 220)

In [4]:
Y.shape

(99990,)

In [5]:
X_comp, X_test, Y_comp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
Xtr, Xva, Ytr, Yva = train_test_split(X_comp, Y_comp, test_size=0.2)

In [6]:
Xtr.shape

(63993, 220)

In [7]:
Xva.shape

(15999, 220)

In [8]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import time

In [20]:
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import time

parameters1 = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.05, 0.075, 0.1,0.2],
    "n_estimators":[10, 100, 250, 500, 1000]
}

parameters4 = {"min_samples_split": [2,6,10,20,40,60],
    "min_samples_leaf": [1,3,5,7,9],}

parameters5 = {"max_features":["log2","sqrt"]}

parameters6 = {"subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0]}
    
GBC = GridSearchCV(GradientBoostingClassifier(), parameters1, cv=5)

print('training start')
starting_time = time.time()
GBC.fit(Xtr, Ytr)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

# p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}

# tuning = GridSearchCV(estimator=GradientBoostingClassifier(max_depth=4, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
#             param_grid = p_test3, scoring='accuracy',n_jobs=4,iid=False, cv=5)
# print('training start')
# tuning.fit(Xtr,Ytr)
# print('trainnig end\n')
# tuning.grid_scores_, tuning.best_params_, tuning.best_score_

training start
training finished, took 22985.771896123886 seconds


In [21]:
gradient_boosting_classifier_roc = roc_auc_score(
   Yva, GBC.predict_proba(Xva)[:,1])
print(gradient_boosting_classifier_roc)

print("training error:", 1 - GBC.score(Xtr, Ytr))
print("validation error:", 1 - GBC.score(Xva, Yva))

0.7600362031854073
training error: 0.28381229196943414
validation error: 0.3077692355772236


In [24]:
best_estimator = GBC.best_estimator_
best_score = GBC.best_score_
best_params = GBC.best_params_
best_index = GBC.best_index_
scorer = GBC.scorer_
cv_results = GBC.cv_results_
refit_time = GBC.refit_time_

In [25]:
best_params

{'learning_rate': 0.1, 'loss': 'deviance', 'n_estimators': 1000}

In [26]:
parameters2 = {
    "loss":["deviance"],
    "learning_rate": [0.1],
    "n_estimators":[1000,1250,1500,1750]
}

GBC2 = GridSearchCV(GradientBoostingClassifier(), parameters2, cv=5)

print('training start')
starting_time = time.time()
GBC2.fit(Xtr, Ytr)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

gradient_boosting_classifier_roc = roc_auc_score(
   Yva, GBC2.predict_proba(Xva)[:,1])
print(gradient_boosting_classifier_roc)

print("training error:", 1 - GBC2.score(Xtr, Ytr))
print("validation error:", 1 - GBC2.score(Xva, Yva))

training start
training finished, took 13299.6862180233 seconds
0.7603106412017687
training error: 0.2769834200615693
validation error: 0.30870679417463587


In [27]:
best_estimator2 = GBC2.best_estimator_
best_score2 = GBC2.best_score_
best_params2 = GBC2.best_params_
best_index2 = GBC2.best_index_
scorer2 = GBC2.scorer_
cv_results2 = GBC2.cv_results_
refit_time2 = GBC2.refit_time_

In [28]:
best_params2

{'learning_rate': 0.1, 'loss': 'deviance', 'n_estimators': 1500}

0.691935055396684

In [36]:
parameters3 = {"max_depth":[3,5,8],
               "loss":["deviance"],
               "learning_rate": [0.1],
               "n_estimators":[1500]
              }

GBC3 = GridSearchCV(GradientBoostingClassifier(), parameters3, cv=5)

print('training start')
starting_time = time.time()
GBC3.fit(Xtr, Ytr)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

gradient_boosting_classifier_roc = roc_auc_score(
   Yva, GBC3.predict_proba(Xva)[:,1])
print(gradient_boosting_classifier_roc)

print("training error:", 1 - GBC3.score(Xtr, Ytr))
print("validation error:", 1 - GBC3.score(Xva, Yva))

best_estimator3 = GBC3.best_estimator_
best_score3 = GBC3.best_score_
best_params3 = GBC3.best_params_
best_index3 = GBC3.best_index_
scorer3 = GBC3.scorer_
cv_results3 = GBC3.cv_results_
refit_time3 = GBC3.refit_time_

training start
training finished, took 21781.42532300949 seconds
0.7603145828608906
training error: 0.27778038222930634
validation error: 0.3093943371460717


In [38]:
best_params3

{'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'n_estimators': 1500}

In [9]:
parameters4 = {
    "max_depth":[3],
    "loss":["deviance"],
    "learning_rate": [0.1],
    "n_estimators":[1500],
    "min_samples_split": [2,6,10,20,40,60],
    "min_samples_leaf": [1,3,5,7,9]}

GBC4 = GridSearchCV(GradientBoostingClassifier(), parameters4, cv=5)

print('training start')
starting_time = time.time()
GBC4.fit(Xtr, Ytr)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

gradient_boosting_classifier_roc = roc_auc_score(
   Yva, GBC4.predict_proba(Xva)[:,1])
print(gradient_boosting_classifier_roc)

print("training error:", 1 - GBC4.score(Xtr, Ytr))
print("validation error:", 1 - GBC4.score(Xva, Yva))

best_estimator4 = GBC4.best_estimator_
best_score4 = GBC4.best_score_
best_params4 = GBC4.best_params_
best_index4 = GBC4.best_index_
scorer4 = GBC4.scorer_
cv_results4 = GBC4.cv_results_
refit_time4 = GBC4.refit_time_

training start


KeyboardInterrupt: 

In [ ]:
print('best_estimator4:\t', best_estimator4)
print('best_score4:\t', best_score4)
print('best_params4:\t', best_params4)
print('best_index4:\t', best_index4)
print('scorer4:\t', scorer4)
print('cv_results4:\t', cv_results4)
print('refit_time4:\t', refit_time4)

In [ ]:
clf = GradientBoostingClassifier(max_depth=3,loss="deviance",learning_rate=0.1,
                                 n_estimators=1500,min_samples_split=,min_samples_leaf=)

parameters5 = {"max_features":["log2","sqrt"]}

GBC5 = GridSearchCV(clf, parameters5, cv=5)

print('training start')
starting_time = time.time()
GBC5.fit(Xtr, Ytr)
end_time = time.time()
print("training finished, took {} seconds".format(end_time - starting_time))

gradient_boosting_classifier_roc = roc_auc_score(
   Yva, GBC5.predict_proba(Xva)[:,1])
print(gradient_boosting_classifier_roc)

print("training error:", 1 - GBC5.score(Xtr, Ytr))
print("validation error:", 1 - GBC5.score(Xva, Yva))

best_estimator5 = GBC5.best_estimator_
best_score5 = GBC5.best_score_
best_params5 = GBC5.best_params_
best_index5 = GBC5.best_index_
scorer5 = GBC5.scorer_
cv_results5 = GBC5.cv_results_
refit_time5 = GBC5.refit_time_


In [ ]:


parameters6 = {"subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0]}